In [103]:
import utils
import Simple_CNN

import matplotlib.pyplot as plt

import torch
import torchsummary
from torchvision import transforms

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import random_split

from torch.utils.data import DataLoader

from sklearn.preprocessing import OneHotEncoder


import numpy as np
from importlib import reload

In [104]:
# Parameters
random_seed = 42

In [105]:
reload(utils)

transform_img = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))
    ]
)

transform_labels = transforms.Compose(
    [transforms.ToTensor()]
)

dataset = utils.SpectrogramDataset("../datasets/Aves-Mammalia.csv", 
                                   "../spectrograms/small_dataset", 
                                   "gbifID", "class", 
                                   transform=transform_img, 
                                   one_hot_encode_labels=True)
sample = dataset[0]

In [106]:
test_percentage = .2
test_len = int((len(dataset) * test_percentage))
train_len = len(dataset) - test_len

train, test = random_split(dataset, [train_len, test_len], torch.Generator().manual_seed(random_seed))

train_loader = DataLoader(train, batch_size=16)
test_loader = DataLoader(test, batch_size=16)

In [110]:
reload(Simple_CNN)
net = Simple_CNN.SimpleCNN(3, [1024, 256], 2)

criterion = nn.BCELoss()
optimizer = optim.Adam(net.parameters(), lr=.001)

net.train_loop(5, train_loader, test_loader, criterion, optimizer)

# torchsummary.summary(net, (3, 1024, 256))

  0%|          | 0/5 [00:00<?, ?it/s]


RuntimeError: Input type (torch.FloatTensor) and weight type (torch.DoubleTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

In [ ]:
print(next(iter(train_loader))["image"].shape)

torch.Size([16, 256, 1024, 3])


In [ ]:
torchsummary.summary(net, (256, 1024, 3))

torch.Size([2, 3, 256, 1024])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1        [-1, 16, 256, 1024]             208
         MaxPool2d-2         [-1, 16, 128, 512]               0
              ReLU-3         [-1, 16, 128, 512]               0
            Conv2d-4         [-1, 32, 128, 512]           2,080
         MaxPool2d-5          [-1, 32, 64, 256]               0
              ReLU-6          [-1, 32, 64, 256]               0
            Conv2d-7          [-1, 64, 64, 256]           8,256
         MaxPool2d-8          [-1, 64, 32, 128]               0
              ReLU-9          [-1, 64, 32, 128]               0
           Conv2d-10          [-1, 32, 32, 128]           8,224
        MaxPool2d-11           [-1, 32, 16, 64]               0
             ReLU-12           [-1, 32, 16, 64]               0
           Conv2d-13           [-1, 16, 16, 64]           2,064
        M

c:\Users\gusta\Git\Animal-Sound\models\Simple_CNN.py:65: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.last_activation(self.dense2(x))
